In [ ]:
## Initialise libaries

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from keras.models import model_from_json
from pylab import plot, show, subplot, specgram, imshow, savefig
import numpy as np
import cv2
import pandas as pd
from shapely.wkt import loads as wkt_loads
import tifffile as tiff
import os, sys
import random
from keras.models import Model
from tensorflow.python.keras import Sequential
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout
from keras.layers import concatenate
#from keras.layers import LeakyReLU
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.advanced_activations import PReLU, ThresholdedReLU
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard
import tensorflow as tf
from keras import backend as K
from sklearn.metrics import jaccard_similarity_score
import skimage.transform
import skimage.morphology
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
from collections import defaultdict
import scipy
from scipy import ndimage
from scipy.ndimage.interpolation import rotate as rotate
import time
#import tempfile
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import keras
import pickle

Using TensorFlow backend.


## Initialies parameters

In [ ]:
img_count = 10   # po kiek pav imsim mokymui 3 - reiskia 3x3; 5 - 5x5
test_img_count = 5
img_size = 1300  # koks apmokymui skirtu pav. dydis pikseliais

N_Cls = 1 #10
N_bands = 3
N_ToPredict = 200

#hyperparamentrai
epochs = 30
train_size=25000
val_size=0.2 #procentide dalis valicacijai
ba_size = 128  # was 128
#def leaky_relu(features, alpha=0.2, name=None) #Compute the Leaky ReLU activation function.
def leaky_relu(x): 
    return LeakyReLU(alpha=0.1)(x)
def prelu(x): 
    return PReLU()(x)
def threshrelu(x): 
    return ThresholdedReLU()(x)

activation_f1 = 'relu'
activation_f1str = 'relu'
activation_f2 = 'sigmoid'
ISZ = 192 
Part_Proc = 0.02
pixel_threshold = 100 
mask_overlap_koef = 0.25
version_ = '4.1.3' #data.model.experiment
Data_Folder = 'AOI_train_125';
weights_spacenet = Data_Folder + '/trained_models_v4/v_'+str(version_)+'_unet_1_jk_' + str(epochs) + 'ep_' + str(train_size) +'siz_'+ str(ba_size)+'ba_'+str(ISZ) + 'ICZ_'+ str(Part_Proc) +'proc';  
model_file = Data_Folder + '/trained_models_v4/v_'+str(version_)+'_model_spacenet_'+ str(epochs) + 'ep_' + str(train_size) +'siz_'+ str(ba_size)+'ba_'+str(ISZ) + 'ICZ_' + str(Part_Proc) +'proc'+'.json'; 
model_pb = Data_Folder + '/trained_models_v4/v_'+str(version_)+'_model_spacenet_'+ str(epochs) + 'ep_' + str(train_size) +'siz_'+ str(ba_size)+'ba_'+str(ISZ) + 'ICZ_' + str(Part_Proc) +'proc'+'.pb';
temp_weights = Data_Folder +'/trained_models_v4/v_'+str(version_)+'_unet_tmp_' +str(epochs) + 'ep_' + str(train_size) +'siz_'+ str(ba_size)+'ba_'+str(ISZ) + 'ICZ_'+ str(Part_Proc) +'proc' +'.hdf5'
Log_Dir = Data_Folder +'/log_data_v4/v_'+str(version_) + '_' + activation_f1str +'_20191216/'
GS_s = pd.read_csv(Data_Folder +'/input_data/grid_sizes_125.csv', names=['ImageId','Xmin', 'Xmax', 'Ymin', 'Ymax'], skiprows=1)
DF_s = pd.read_csv(Data_Folder +'/input_data/combined_polygons_125_v4.csv')

Big_x = Data_Folder + '/composed_data_v4/all_x_trn_3_%d'
Big_y = Data_Folder + '/composed_data_v4/all_y_trn_3_%d'

Big_test_x = Data_Folder + '/composed_data_v4/all_x_test_3_%d'
Big_test_y = Data_Folder + '/composed_data_v4/all_y_test_3_%d'

patches_file_x = Data_Folder +'/composed_data_v4/x_data_v_4_patches_v_6_ISZ192_25000.npz'
patches_file_y = Data_Folder +'/composed_data_v4/y_data_v_4_patches_v_6_ISZ192_25000.npz'

smooth = 1e-12
print("done")

done


## Define all functions

In [ ]:
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print ('%2.2f' % \
                 ((te - ts) * 1000,))
        return result
    return timed

def _convert_coordinates_to_raster_spacenet(coords, img_size, xyminmax):
    Xmin, Xmax, Ymin, Ymax = xyminmax
    H, W = img_size
    coords[:, 1] = H - (coords[:, 1] - Ymin)/(Ymax-Ymin)*H
    coords[:, 0] = (coords[:, 0] - Xmin)/(Xmax-Xmin)*W
    coords_int = np.round(coords).astype(np.int32)
    return coords_int


def _get_xmin_xmax_ymin_ymax_spacenet(grid_sizes_panda, imageId):
    xmin, xmax, ymin, ymax = grid_sizes_panda[grid_sizes_panda.ImageId == imageId].iloc[0, 1:].astype(float)
    return (xmin,xmax, ymin, ymax)


def _get_polygon_list(wkt_list_pandas, imageId, cType):
    df_image = wkt_list_pandas[wkt_list_pandas.ImageId == imageId]
    multipoly_def = df_image[df_image.ClassType == cType].MultipolygonWKT
    polygonList = None
    if len(multipoly_def) > 0:
        assert len(multipoly_def) == 1
        polygonList = wkt_loads(multipoly_def.values[0])
    return polygonList

def _get_and_convert_contours_spacenet(polygonList, raster_img_size, xyminmax):
    perim_list = []
    interior_list = []
    if polygonList is None:
        return None
    for k in range(len(polygonList)):
        poly = polygonList[k]
        perim = np.array(list(poly.exterior.coords))
        perim_c = _convert_coordinates_to_raster_spacenet(perim, raster_img_size, xyminmax)
        if not any(np.array_equal(perim_c, x) for x in perim_list):
            perim_list.append(perim_c)
            for pi in poly.interiors:
                interior = np.array(list(pi.coords))
                interior_c = _convert_coordinates_to_raster_spacenet(interior, raster_img_size, xyminmax)
                interior_list.append(interior_c)
    return perim_list, interior_list

def _plot_mask_from_contours(raster_img_size, contours, class_value=1):
    img_mask = np.zeros(raster_img_size, np.uint8)
    if contours is None:
        return img_mask
    perim_list, interior_list = contours
    cv2.fillPoly(img_mask, perim_list, class_value)
    cv2.fillPoly(img_mask, interior_list, 0)
    return img_mask


def generate_mask_for_image_and_class_spacenet(raster_size, imageId, class_type, grid_sizes_panda=GS_s, wkt_list_pandas=DF_s):
    xyminmax = _get_xmin_xmax_ymin_ymax_spacenet(grid_sizes_panda, imageId)
    polygon_list = _get_polygon_list(wkt_list_pandas, imageId, class_type)
    contours = _get_and_convert_contours_spacenet(polygon_list, raster_size, xyminmax)
    mask = _plot_mask_from_contours(raster_size, contours, 1)
    return mask


def M_spacenet(image_id):
    img = tiff.imread('AOI_train_125/input_data/{}.tif'.format(image_id))
    img = cv2.resize(img, (img_size, img_size))
    return img

def M_spacenet_small(image_id):
    img = tiff.imread('AOI_train_125/input_data/{}.tif'.format(image_id))
    img = cv2.resize(img, (ISZ, ISZ))
    return img


def stretch_n(bands, lower_percent=0, higher_percent=100):
    out = np.zeros_like(bands, dtype=np.float32)
    n = bands.shape[2]
    for i in range(n):
        a = 0
        b = 1
        c = np.percentile(bands[:, :, i], lower_percent)
        d = np.percentile(bands[:, :, i], higher_percent)
        t = a + (bands[:, :, i] - c) * (b - a) / (d - c)
        t[t < a] = a
        t[t > b] = b
        out[:, :, i] = t

    return out.astype(np.float32)


def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    rez=K.mean(jac)
    return rez


def jaccard_coef_int(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    rez=K.mean(jac)
    return rez


def stick_all_train_spacenet():
    print("Stick all imgs together:")
    s = img_size

    x = np.zeros((img_count * s, img_count * s, N_bands))
    y = np.zeros((img_count * s, img_count * s, N_Cls))
    
    x_test = np.zeros((test_img_count * s, test_img_count * s, N_bands))
    y_test = np.zeros((test_img_count * s, test_img_count * s, N_Cls))
    
    ids_all = sorted(DF_s.ImageId.unique())
    random.shuffle(ids_all)
    ids = ids_all[:img_count*img_count]
    ids_test = ids_all[-(test_img_count*test_img_count):]
    del ids_all
    
    for i in range(img_count):
        for j in range(img_count):
            id_ = ids[img_count * i + j]
            print("train id", id_)
            img = M_spacenet(id_)
            img = stretch_n(img)
            x[s * i:s * i + s, s * j:s * j + s, :] = img[:s, :s, :]
            for z in range(N_Cls):
                y[s * i:s * i + s, s * j:s * j + s, z] = generate_mask_for_image_and_class_spacenet(
                    (img.shape[0], img.shape[1]), id_, z + 1)[:s, :s]
    np.save(Big_x % N_Cls, x)
    np.save(Big_y % N_Cls, y)
    
    for i in range(test_img_count):
        for j in range(test_img_count):
            id_ = ids_test[test_img_count * i + j]
            print("test id", id_)
            img = M_spacenet(id_)
            img = stretch_n(img)
            x_test[s * i:s * i + s, s * j:s * j + s, :] = img[:s, :s, :]
            for z in range(N_Cls):
                y_test[s * i:s * i + s, s * j:s * j + s, z] = generate_mask_for_image_and_class_spacenet(
                    (img.shape[0], img.shape[1]), id_, z + 1)[:s, :s]
    
    np.save(Big_test_x % N_Cls, x_test)
    np.save(Big_test_y % N_Cls, y_test)

    
def stick_all_train_spacenet_v2():
    print("Stick all imgs together:")
    sx = img_size
    sy = img_size
    x = np.zeros((img_count * sx, img_count * sy, N_bands))
    y = np.zeros((img_count * sx, img_count * sy, N_Cls))
    
    x_test = np.zeros((test_img_count * sx, test_img_count * sy, N_bands))
    y_test = np.zeros((test_img_count * sx, test_img_count * sy, N_Cls))
    
    ids_all = sorted(DF_s.ImageId.unique())
    random.shuffle(ids_all)
    print(ids_all)
    counter = 0
    for id_ in ids_all:
        counter += 1
        img = M_spacenet(id_)
        img = stretch_n(img)
        if counter <= 50:
            s = img_count * 2
            sy = int( img_size / 2 )
            i = int((counter - 1) / s)
            j = ((counter - 1) % s)
        else:
            s = img_count
            sy = img_size
            i = int((counter - 16) / s) - 1
            j = (counter + 4) % s
        x[sx * i:sx * i + sx, sy * j:sy * j + sy, :] = img[:sx, :sy, :]    
        for z in range(N_Cls):
            y[sx * i:sx * i + sx, sy * j:sy * j + sy, z] = generate_mask_for_image_and_class_spacenet(
                (img.shape[0], img.shape[1]), id_, z + 1)[:sx, :sy]
        
        if counter <= 50:
            s = test_img_count * 2
            sy = int( img_size / 2 )
            i = int((counter - 1) / s)
            j = ((counter - 1) % s)
            x_test[sx * i:sx * i + sx, sy * j:sy * j + sy, :] = img[:sx, sy:, :]
            for z in range(N_Cls):
                y_test[sx * i:sx * i + sx, sy * j:sy * j + sy, z] = generate_mask_for_image_and_class_spacenet(
                    (img.shape[0], img.shape[1]), id_, z + 1)[:sx, sy:] 
    del ids_all
    np.save(Big_x % N_Cls, x)
    np.save(Big_y % N_Cls, y)
    np.save(Big_test_x % N_Cls, x_test)
    np.save(Big_test_y % N_Cls, y_test)
    
def random_brightness(image):
    image = cv2.cvtColor(image.astype(np.float32), cv2.COLOR_BGR2RGB)    
    # Convert 2 HSV colorspace from BGR colorspace
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Generate new random brightness
    rand = random.uniform(0.5, 1.0)
    hsv[:, :, 2] = rand*hsv[:, :, 2]
    # Convert back to BGR colorspace
    new_img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return new_img


def rotate_image(img, angle, pivot):
    padX = [img.shape[1] - pivot[0], pivot[0]]
    padY = [img.shape[0] - pivot[1], pivot[1]]
    imgP = np.pad(img, [padY, padX, [0, 0]], 'constant')
    imgR = ndimage.rotate(imgP, angle, reshape=False)
    return imgR[padY[0] : -padY[1], padX[0] : -padX[1]]


def rotate_patch_image(img, msk):
    pivot = (int(ISZ/2), int(ISZ/2))
    angle = random.randrange(1, 360, 1)
    rot_img = rotate_image(img, angle, pivot)
    rot_msk = rotate_image(msk, angle, pivot)
    return rot_img, rot_msk


def add_noise(img):
    # ADDING NOISE
    noise = np.random.uniform(-0.05, 0.05, size = (img.shape[0], img.shape[1], 3))
    img = img + noise
    img[img>1] = 1
    img[img<0] = 0
    return img


def add_noise_to_color(img):
    noise = np.random.uniform(-0.15, 0.15)
    ch = np.random.randint(0,3)
    img[:,:,ch] = img[:,:,ch] + noise
    img[img>1] = 1
    img[img<0] = 0
    return img


def validate_coord(xc,yc):
    is2 = int(1.0 * ISZ)
    if xc < img_size * 2:
        if img_size - (xc % img_size) < is2:
            return False
        if ((img_size/2) - (yc % (img_size/2))) < is2:
            return False
    elif xc > img_size * 2 and xc < img_size * 3 and yc < img_size * 5:
        if img_size - (xc % img_size) < is2:
            return False
        if ((img_size/2) - (yc % (img_size/2))) < is2:
            return False
    else:    
        if img_size - (xc % img_size) < is2:
            return False
        elif img_size - (yc % img_size) < is2:
            return False
    
    return True

def get_x_patches_from_file():
    print("Loading x patches from file")
    data1 = np.load(patches_file_x)
    return data1['x']

def get_patches_from_file():
    print("Loading patches from file")
    data1 = np.load(patches_file_x)
    data2 = np.load(patches_file_y)
    return data1['x'], data2['y']
    
def get_patches_spacenet(img, msk, amt, aug=True, show_image=False):
    is2 = int(1.0 * ISZ)
    xm, ym = img.shape[0], img.shape[1]

    if show_image:
        fig, ax1 = plt.subplots(figsize=(60,40),facecolor='w')
        ax1.set_title('Big Image')
        plt.imshow(msk[:,:,0], cmap=plt.get_cmap('gray'))
    
    x, y = [], []
    coords_arr = []
    coords_arr2 = []
    skipped = 0
    tr = [Part_Proc]
    skip_limit = 15000
    while len(x) < amt and skipped < skip_limit:
        xc = random.randint(0, xm)
        yc = random.randint(0, ym)
        if not validate_coord(xc,yc):
            skipped += 1
            continue
            
        coord = str(xc)+"_"+str(yc)
        if coord in coords_arr:
            skipped += 1
            print ("Total skipped images:", skipped, " Total:", len(x))
            continue
        else:
            coords_arr.append(coord) 

        im = img[xc:xc + is2, yc:yc + is2]
        ms = msk[xc:xc + is2, yc:yc + is2]
        for j in range(N_Cls):        
            sm = np.sum(ms[:, :, j])
            if ((1.0 * sm) / (is2 ** 2)) > tr[j]: 
                coords_arr2.append(coord) 
                x.append(im.astype(np.float32))
                y.append(ms.astype(np.float32))
                
                if aug and (len(x) + 6 < amt):
                    if random.uniform(0, 1) > 0.9:
                        im_aug = im[::-1]
                        ms_aug = ms[::-1]
                        x.append(im_aug.astype(np.float32))
                        y.append(ms_aug.astype(np.float32))
                    if random.uniform(0, 1) > 0.9:
                        im_aug = im[:, ::-1]
                        ms_aug = ms[:, ::-1]
                        x.append(im_aug.astype(np.float32))
                        y.append(ms_aug.astype(np.float32))
                    if random.uniform(0, 1) > 0.7:
                        im_aug = random_brightness(im)
                        x.append(im_aug.astype(np.float32))
                        y.append(ms.astype(np.float32))                        
                    if random.uniform(0, 1) > 0.7:    
                        im_aug = add_noise(im)
                        x.append(im_aug.astype(np.float32))
                        y.append(ms.astype(np.float32))
                    if random.uniform(0, 1) > 0.8:    
                        im_aug = add_noise_to_color(im)
                        x.append(im_aug.astype(np.float32))
                        y.append(ms.astype(np.float32))
                    if random.uniform(0, 1) > 1:
                        im_aug, ms_aug = rotate_patch_image(im, ms)
                        im_aug = stretch_n(im_aug)
                        ms_aug = np.round(ms_aug, 0)
                        x.append(im_aug.astype(np.float32))
                        y.append(ms_aug.astype(np.float32))

                if show_image:
                    rect = patches.Rectangle((yc,xc), is2, is2,linewidth=2,edgecolor='r',facecolor='none', fill=False)
                    ax1.add_patch(rect)
                    
                if len(x) % 100 == 0:
                    print(" Working: ", len(x), "/", amt)
    if show_image:
        plt.show() 
    
    x, y = 2 * np.transpose(x, (0, 3, 1, 2)) - 1, np.transpose(y, (0, 3, 1, 2))
    print(x.shape, y.shape, np.amax(x), np.amin(x), np.amax(y), np.amin(y), type(x[0]), type(np.amin(x)))
    return x, y 



def make_val_spacenet():
    print("let's pick some samples for validation")
    img = np.load('AOI_train_81/composed_data/x_trn_3_%d.npy' % N_Cls)
    msk = np.load('AOI_train_81/composed_data/y_trn_3_%d.npy' % N_Cls)
    x, y = get_patches_spacenet(img, msk, amt=3000)

    np.save('AOI_train_81/composed_data/x_tmp_3_%d'  % N_Cls, x)
    np.save('AOI_train_81/composed_data/y_tmp_3_%d'  % N_Cls, y)


def get_unet_2_old():
    inputs = Input((N_bands, ISZ, ISZ))
    conv1 = Conv2D(8, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(inputs)
    conv1 = Conv2D(8, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv1)

    conv2 = Conv2D(16, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool1)
    conv2 = Conv2D(16, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv2)

    conv3 = Conv2D(32, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool2)
    conv3 = Conv2D(32, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv3)

    up8 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv3), conv2], axis=1)
    conv8 = Conv2D(16, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up8)
    conv8 = Conv2D(16, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv8), conv1], axis=1)
    conv9 = Conv2D(8, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up9)
    conv9 = Conv2D(8, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv9)

    conv10 = Conv2D(N_Cls, (1, 1), activation=activation_f2, data_format="channels_first")(conv9)

    model = Model(inputs=inputs, outputs=conv10)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[jaccard_coef, jaccard_coef_int, 'accuracy'])
    return model


def get_unet_1():
    inputs = Input((N_bands, ISZ, ISZ))
    conv1 = Conv2D(32, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(inputs)
    conv1 = Conv2D(32, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv1)

    conv2 = Conv2D(64, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool1)
    conv2 = Conv2D(64, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv2)

    conv3 = Conv2D(128, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool2)
    conv3 = Conv2D(128, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv3)

    conv4 = Conv2D(256, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool3)
    conv4 = Conv2D(256, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv4)
    
    up5 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv4), conv3], axis=1)
    conv5 = Conv2D(128, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up5)
    conv5 = Conv2D(128, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv5)

    up6 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv5), conv2], axis=1)
    conv6 = Conv2D(64, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up6)
    conv6 = Conv2D(64, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv6), conv1], axis=1)
    conv7 = Conv2D(32, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up7)
    conv7 = Conv2D(32, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv7)

    conv8 = Conv2D(N_Cls, (1, 1), activation=activation_f2, data_format="channels_first")(conv7)

    model = Model(inputs=inputs, outputs=conv8)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[jaccard_coef, jaccard_coef_int, 'accuracy'])
    return model


def get_unet_2():
    inputs = Input((N_bands, ISZ, ISZ))
    conv1 = Conv2D(32, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(inputs)
    conv1 = Conv2D(32, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv1)

    conv2 = Conv2D(64, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool1)
    conv2 = Conv2D(64, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv2)

    conv3 = Conv2D(128, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool2)
    conv3 = Conv2D(128, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv3)

    conv4 = Conv2D(256, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool3)
    conv4 = Conv2D(256, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv4)

    conv5 = Conv2D(512, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool4)
    conv5 = Conv2D(512, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv5)
   
    up6 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv5), conv4], axis=1)
    conv6 = Conv2D(256, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up6)
    conv6 = Conv2D(256, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv6)

    up7 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv6), conv3], axis=1)
    conv7 = Conv2D(128, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up7)
    conv7 = Conv2D(128, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv7)

    up8 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv7), conv2], axis=1)
    conv8 = Conv2D(64, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up8)
    conv8 = Conv2D(64, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv8), conv1], axis=1)
    conv9 = Conv2D(32, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up9)
    conv9 = Conv2D(32, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv9)

    conv10 = Conv2D(N_Cls, (1, 1), activation=activation_f2, data_format="channels_first")(conv9)

    model = Model(inputs=inputs, outputs=conv10)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[jaccard_coef, jaccard_coef_int, 'accuracy'])
    return model

def get_unet_3():
    inputs = Input((N_bands, ISZ, ISZ))
    conv1 = Conv2D(32, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(inputs)
    conv1 = Conv2D(32, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv1)

    conv2 = Conv2D(64, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool1)
    conv2 = Conv2D(64, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv2)

    conv3 = Conv2D(128, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool2)
    conv3 = Conv2D(128, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv3)

    conv4 = Conv2D(256, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool3)
    conv4 = Conv2D(256, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv4)

    conv5 = Conv2D(512, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool4)
    conv5 = Conv2D(512, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv5)
    pool5 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv5)   
        
    conv6 = Conv2D(1024, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool5)
    conv6 = Conv2D(1024, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv6)
    
    up7 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv6), conv5], axis=1)
    conv7 = Conv2D(512, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up7)
    conv7 = Conv2D(512, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv7) 
   
    up8 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv7), conv4], axis=1)
    conv8 = Conv2D(256, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up8)
    conv8 = Conv2D(256, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv8)

    up9 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv8), conv3], axis=1)
    conv9 = Conv2D(128, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up9)
    conv9 = Conv2D(128, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv9)

    up10 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv9), conv2], axis=1)
    conv10 = Conv2D(64, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up10)
    conv10 = Conv2D(64, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv10)

    up11 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv10), conv1], axis=1)
    conv11 = Conv2D(32, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up11)
    conv11 = Conv2D(32, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv11)

    conv12 = Conv2D(N_Cls, (1, 1), activation=activation_f2, data_format="channels_first")(conv11)

    model = Model(inputs=inputs, outputs=conv12)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[jaccard_coef, jaccard_coef_int, 'accuracy'])
    return model

def get_unet_4():
    inputs = Input((N_bands, ISZ, ISZ))
    conv1 = Conv2D(32, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(inputs)
    conv1 = Conv2D(32, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv1)

    conv2 = Conv2D(64, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool1)
    conv2 = Conv2D(64, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv2)

    conv3 = Conv2D(128, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool2)
    conv3 = Conv2D(128, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv3)

    conv4 = Conv2D(256, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool3)
    conv4 = Conv2D(256, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv4)

    conv5 = Conv2D(512, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool4)
    conv5 = Conv2D(512, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv5)
    pool5 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv5)   
 
    conv6 = Conv2D(1024, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool5)
    conv6 = Conv2D(1024, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv6)
    pool6 = MaxPooling2D(pool_size=(2, 2), data_format="channels_first")(conv6)        
    
    conv7 = Conv2D(2048, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(pool6)
    conv7 = Conv2D(2048, (3, 3), activation=activation_f1, padding='same', data_format="channels_first")(conv7)
    
    up8 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv7), conv6], axis=1)
    conv8 = Conv2D(1024, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up8)
    conv8 = Conv2D(1024, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv8)    
    
    up9 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv8), conv5], axis=1)
    conv9 = Conv2D(512, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up9)
    conv9 = Conv2D(512, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv9) 
   
    up10 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv9), conv4], axis=1)
    conv10 = Conv2D(256, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up10)
    conv10 = Conv2D(256, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv10)

    up11 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv10), conv3], axis=1)
    conv11 = Conv2D(128, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up11)
    conv11 = Conv2D(128, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv11)

    up12 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv11), conv2], axis=1)
    conv12 = Conv2D(64, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up12)
    conv12 = Conv2D(64, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv12)

    up13 = concatenate([UpSampling2D(size=(2, 2), data_format="channels_first")(conv12), conv1], axis=1)
    conv13 = Conv2D(32, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(up13)
    conv13 = Conv2D(32, (3, 3), activation=activation_f1, padding='same',  data_format="channels_first")(conv13)

    conv14 = Conv2D(N_Cls, (1, 1), activation=activation_f2, data_format="channels_first")(conv13)

    model = Model(inputs=inputs, outputs=conv14)
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=[jaccard_coef, jaccard_coef_int, 'accuracy'])
    return model

def calc_jacc(model):
    img = np.load('data/x_tmp_3_%d.npy' % N_Cls)
    msk = np.load('data/y_tmp_3_%d.npy' % N_Cls)

    prd = model.predict(img, batch_size=4)
    print(prd.shape, msk.shape)
    avg, trs = [], []

    for i in range(N_Cls):
        t_msk = msk[:, i, :, :]
        t_prd = prd[:, i, :, :]
        t_msk = t_msk.reshape(msk.shape[0] * msk.shape[2], msk.shape[3])
        t_prd = t_prd.reshape(msk.shape[0] * msk.shape[2], msk.shape[3])

        m, b_tr = 0, 0
        for j in range(10):
            tr = j / 10.0
            pred_binary_mask = t_prd > tr

            jk = jaccard_similarity_score(t_msk, pred_binary_mask)
            if jk > m:
                m = jk
                b_tr = tr
        print(i, m, b_tr)
        avg.append(m)
        trs.append(b_tr)

    score = sum(avg) / 10.0
    return score, trs


def calc_jacc_spacenet(model):
    img = np.load('composed_data_v4/x_tmp_3_%d.npy' % N_Cls)
    msk = np.load('composed_data_v4/y_tmp_3_%d.npy' % N_Cls)

    prd = model.predict(img, batch_size=4)
    avg, trs = [], []

    for i in range(N_Cls):
        t_msk = msk[:, i, :, :]
        t_prd = prd[:, i, :, :]
        t_msk = t_msk.reshape(msk.shape[0] * msk.shape[2], msk.shape[3])
        t_prd = t_prd.reshape(msk.shape[0] * msk.shape[2], msk.shape[3])

        m, b_tr = 0, 0
        for j in range(10):
            tr = j / 10.0
            pred_binary_mask = t_prd > tr

            jk = jaccard_similarity_score(t_msk, pred_binary_mask)
            if jk > m:
                m = jk
                b_tr = tr
        print(i, m, b_tr)
        avg.append(m)
        trs.append(b_tr)

    score = sum(avg) / 10.0
    return score, trs



def mask_for_polygons(polygons, im_size):
    img_mask = np.zeros(im_size, np.uint8)
    if not polygons:
        return img_mask
    int_coords = lambda x: np.array(x).round().astype(np.int32)
    exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
    interiors = [int_coords(pi.coords) for poly in polygons
                 for pi in poly.interiors]
    cv2.fillPoly(img_mask, exteriors, 1)
    cv2.fillPoly(img_mask, interiors, 0)
    return img_mask


def mask_to_polygons(mask, epsilon=1, min_area=1.):
    image, contours, hierarchy = cv2.findContours(
        ((mask == 1) * 255).astype(np.uint8),
        cv2.RETR_CCOMP, cv2.CHAIN_APPROX_TC89_KCOS)
    approx_contours = [cv2.approxPolyDP(cnt, epsilon, True)
                       for cnt in contours]
    if not contours:
        return MultiPolygon()
    cnt_children = defaultdict(list)
    child_contours = set()
    assert hierarchy.shape[0] == 1
    for idx, (_, _, _, parent_idx) in enumerate(hierarchy[0]):
        if parent_idx != -1:
            child_contours.add(idx)
            cnt_children[parent_idx].append(approx_contours[idx])
    all_polygons = []
    for idx, cnt in enumerate(approx_contours):
        if idx not in child_contours and cv2.contourArea(cnt) >= min_area:
            assert cnt.shape[1] == 1
            poly = Polygon(
                shell=cnt[:, 0, :],
                holes=[c[:, 0, :] for c in cnt_children.get(idx, [])
                       if cv2.contourArea(c) >= min_area])
            all_polygons.append(poly)
    all_polygons = MultiPolygon(all_polygons)
    if not all_polygons.is_valid:
        all_polygons = all_polygons.buffer(0)
        if all_polygons.type == 'Polygon':
            all_polygons = MultiPolygon([all_polygons])
    return all_polygons

def train_net_spacenet():
    print("start train net: get patches")
    x_trn, y_trn = get_patches_from_file()
    print('Patches collected:')
    model = get_unet_4()
    print("start train net: start fit")
    model_checkpoint = ModelCheckpoint(temp_weights, monitor='loss', save_best_only=True)
    model_TensorBoard = TensorBoard(log_dir=Log_Dir, histogram_freq=0, batch_size=ba_size, write_graph=False, write_grads=False,
                                    write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, 
                                    embeddings_data=None)

    for i in range(1):
        model.fit(x_trn, y_trn, validation_split=val_size, batch_size=ba_size, epochs=epochs, verbose=1,
                  callbacks=[model_checkpoint, model_TensorBoard], shuffle=True)
        del x_trn
        del y_trn
    return model


def predict_id_spacenet(id, model):
    img = M_spacenet(id)
    x = stretch_n(img)

    cnv = np.zeros((img_size, img_size, N_bands)).astype(np.float32)
    prd = np.zeros((N_Cls, img_size, img_size)).astype(np.float32)
    cnv[:img.shape[0], :img.shape[1], :] = x

    for i in range(0, 8):
        line = []
        for j in range(0, 8):
            line.append(cnv[i * ISZ:(i + 1) * ISZ, j * ISZ:(j + 1) * ISZ])

        x = 2 * np.transpose(line, (0, 3, 1, 2)) - 1
        tmp = model.predict(x, batch_size=4)
        for j in range(tmp.shape[0]):
            prd[:, i * ISZ:(i + 1) * ISZ, j * ISZ:(j + 1) * ISZ] = tmp[j]

    return prd[:, :img.shape[0], :img.shape[1]]

def model_save_spacenet(model_send):
    model_json = model_send.to_json()
    json_file = open(model_file, "w")
    json_file.write(model_json)
    json_file.close()
    model_send.save_weights(weights_spacenet)

def model_load_spacenet():
    json_file = open(model_file, "r")
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json, custom_objects={'prelu': activation_f1})
    loaded_model.load_weights(weights_spacenet)
    return loaded_model

def calculate_mask_count(im):
    labels,number_of_masks = ndimage.label(im)
    areas = ndimage.sum(im, labels, range(1, labels.max()+1))
    areas_threshold = np.where(areas > pixel_threshold)
    areas_threshold = np.asarray(areas_threshold)
    number_of_masks_th=areas_threshold.size
    return (number_of_masks, number_of_masks_th)

def compare_every_mask(pred_mask, true_mask):
    labels,number_of_masks = ndimage.label(pred_mask)
    slices = ndimage.find_objects(labels)

    masks_nm = 0
    found_objs = 0
    for sl in slices:
        sl_size = (pred_mask[sl] == 1).sum() 
        if sl_size >= pixel_threshold:
            masks_nm += 1;
            true_size = (true_mask[sl] == 1).sum()
            if true_size > sl_size * mask_overlap_koef:
                found_objs += 1
                
    print (found_objs, "/", masks_nm, "/", number_of_masks)
    
    return (1,1)

def check_predict_spacenet(img_id,model):
    print('\start predict!!')
    msk = predict_id_spacenet(img_id, model)
    img=M_spacenet(img_id)
    msk_true = np.zeros((img.shape[0], img.shape[1], N_Cls))
    msk_true = generate_mask_for_image_and_class_spacenet((img_size, img_size), img_id, 1)
   
    print("check_predict_spacenet... 1 Mask shapes: ", msk_true.shape, msk.shape)

    true_msk_count_no_thres, true_msk_count = calculate_mask_count(msk_true)
    print('Number of true masks before threshold: ',true_msk_count_no_thres) 
    print('Number of true masks after threshold: ',true_msk_count_no_thres) 
    msk = np.squeeze(msk)
    msk = np.round(msk, 0)
    msk = remove_bad_pixels_from_mask(msk)
    pred_msk_count_no_tres, pred_msk_count = calculate_mask_count(msk)
    print('Number of predicted masks before threshold: ',pred_msk_count_no_tres) 
    print('Number of predicted  masks after threshold: ',pred_msk_count) 
          
    plt.figure()
    fig, ax1 = plt.subplots(figsize=(30,15),facecolor='w')  
    ax1 = plt.subplot(131)
    ax1.set_title('IMAGE')
    ax2 = plt.subplot(132)
    ax2.set_title('predict bldg pixels')
    ax2.imshow(msk)
    rez=jaccard_coef(msk_true, msk)
    print('jaccard_coef: ',rez)

    ax3 = plt.subplot(133)
    ax3.set_title('true mask from file')
    ax3.imshow(msk_true)
    plt.show()
    
    fig, ax1 = plt.subplots(figsize=(60,40),facecolor='w')
    plt.imshow(msk, cmap=plt.get_cmap('gray'))
  
    
def show_image_true_predicted_mask(img_id):  #function for showing and testing
      # --- image and true masks (begin) ---- 
    img = tiff.imread(Data_Folder+'/input_data/{}.tif'.format(img_id))
    img = stretch_n(img)
    msk_true = np.zeros((img.shape[0], img.shape[1], N_Cls))
    msk_true = generate_mask_for_image_and_class_spacenet((img_size, img_size), img_id, 1)
    print('img.size',img.shape[0]) 
    img_and_msk_true = img
    for i in range(img.shape[0]):
      for j in range(img.shape[1]):
        if msk_true[i,j] == 1:
            img_and_msk_true[i,j] = (1, 1 ,1)   
    
    fig, ax1 = plt.subplots(figsize=(60,40),facecolor='w')
    plt.imshow(img_and_msk_true)
    # --- image and true masks (end) ---- 
    
    # --- image and predicted masks (begin) ----
    model = model_load_spacenet()
    msk_pred = predict_id_spacenet(img_id, model)
    msk_pred = np.squeeze(msk_pred)
    msk_pred = np.round(msk_pred, 0)
    print('img.size',img.shape[0]) 
    img_and_msk_pred = img
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if msk_pred[i,j] == 1:
                img_and_msk_pred[i,j] = (1, 1 ,1)   
    
    fig, ax1 = plt.subplots(figsize=(60,40),facecolor='w')
    plt.imshow(img_and_msk_pred)
    # --- image and predicted masks (end) ----  

    # --- image and predicted masks after pixel_threhold(begin) ----
    model = model_load_spacenet()
    msk_pred = predict_id_spacenet(img_id, model)
    msk_pred = np.squeeze(msk_pred)
    msk_pred = np.round(msk_pred, 0)
    msk_pred = remove_bad_pixels_from_mask(msk_pred)
    print('img.size',img.shape[0]) 
    img_and_msk_pred_thres = img
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if msk_pred[i,j] == 1:
                img_and_msk_pred_thres[i,j] = (1, 1 ,1)   
    
    fig, ax1 = plt.subplots(figsize=(60,40),facecolor='w')
    plt.imshow(img_and_msk_pred_thres)
  # --- image and predicted masks (end) ----  


def show_true_and_predicted_mask(img_id):  #function for showing and testing
      # --- image and true masks (begin) ---- 
    img = tiff.imread(Data_Folder+'/input_data/{}.tif'.format(img_id))
    img = stretch_n(img)
    msk_true = np.zeros((img.shape[0], img.shape[1], N_Cls))
    msk_true = generate_mask_for_image_and_class_spacenet((img_size, img_size), img_id, 1)
    fig, ax1 = plt.subplots(figsize=(60,40),facecolor='w')
    plt.imshow(msk_true)
    # --- image and true masks (end) ---- 
    
    # --- image and predicted masks (begin) ----
    model = model_load_spacenet()
    msk_pred = predict_id_spacenet(img_id, model)
    msk_pred = np.squeeze(msk_pred)
    msk_pred = np.round(msk_pred, 0)
       
    fig, ax1 = plt.subplots(figsize=(60,40),facecolor='w')
    plt.imshow(msk_pred)
    # --- image and predicted masks (end) ----  

    # --- image and predicted masks after pixel_threhold(begin) ----
    msk_pred = remove_bad_pixels_from_mask(msk_pred)    
    fig, ax1 = plt.subplots(figsize=(60,40),facecolor='w')
    plt.imshow(msk_pred)
    # --- image and predicted masks (end) ----    
    

def remove_bad_pixels_from_mask(msk):
    kernel = np.ones((5,5))
    msk_new = cv2.morphologyEx(msk, cv2.MORPH_OPEN, kernel)
    return msk_new

def predict_image(img_id):
    img = tiff.imread(Data_Folder+'/input_data/{}.tif'.format(img_id))
    img = stretch_n(img)
    model = model_load_spacenet()
    msk_pred = predict_id_spacenet(img_id, model)
    msk_pred = np.squeeze(msk_pred)
    msk_pred = np.round(msk_pred, 0)
    msk_pred = remove_bad_pixels_from_mask(msk_pred)
    msk_count = calculate_mask_count(msk_pred)
    return msk_count 

def predict_test_image_spacenet(img, model):
    ISZ_ = int(ISZ)
    xx = int((img.shape[0]/ISZ_)-1)
    yy = int((img.shape[1]/ISZ_)-1)
    prd = np.zeros((N_Cls, img.shape[0], 
                           img.shape[1])).astype(np.float32)
                    
                   
    for i in range(0, int(img.shape[0]/ISZ_)):
        line = []
        for j in range(0, int(img.shape[1]/ISZ_)):
            x1 = i * ISZ_
            x2 = (i + 1) * ISZ_
            y1 = j * ISZ_
            y2 = (j + 1) * ISZ_
            line.append(img[x1:x2, y1:y2])
            if i < xx and j == yy:
                y1 = img.shape[1]-ISZ_
                y2 = img.shape[1]
                line.append(img[x1:x2, y1:y2])
                
        x = 2 * np.transpose(line, (0, 3, 1, 2)) - 1
        tmp = model.predict(x, batch_size=4)
        if len(tmp)>img.shape[1]/ISZ:
            last = tmp[-1]
            tmp = tmp[:-1]
        for j in range(tmp.shape[0]):
            prd[:, i * ISZ_:(i + 1) * ISZ_, j * ISZ_:(j + 1) * ISZ_] = tmp[j]
            if i < xx and j == yy:
                prd[:, i * ISZ_:(i + 1) * ISZ_, img.shape[1]-ISZ_:img.shape[1]] = last
                
                
        if i == xx:
            line = []
            for j in range(0, int(img.shape[1]/ISZ_)):
                x1 = img.shape[0]-ISZ_
                x2 = img.shape[0]
                y1 = j * ISZ_
                y2 = (j + 1) * ISZ_
                line.append(img[x1:x2, y1:y2])
                if j==yy:
                    y1 = img.shape[1]-ISZ_
                    y2 = img.shape[1]
                    line.append(img[x1:x2, y1:y2])
            x = 2 * np.transpose(line, (0, 3, 1, 2)) - 1
            tmp = model.predict(x, batch_size=4)
            if len(tmp)>img.shape[1]/ISZ:
                last = tmp[-1]
                tmp = tmp[:-1]
            for j in range(tmp.shape[0]): #
                prd[:, img.shape[0]-ISZ_:img.shape[0], j * ISZ_:(j + 1) * ISZ_] = tmp[j]
                if j == yy:
                    prd[:, img.shape[0]-ISZ_:img.shape[0], img.shape[1]-ISZ_:img.shape[1]] = last
                
                
    return prd[:, :img.shape[0], :img.shape[1]]

@timeit
def model_predict(model,x):
    tmp = model.predict(x, batch_size=128)
    return tmp

#@timeit
def predict_test_image_spacenet_sliding(img, model):
    ISZ_ = int(ISZ)
    step = int(ISZ_/2)
    xx = int((img.shape[0]/step)-2)
    yy = int((img.shape[1]/step)-2)
    prd = np.zeros((N_Cls, img.shape[0], 
                           img.shape[1])).astype(np.float32)

    for i in range(0, int( img.shape[0]/step )-1):
        line = []
        for j in range(0, int( img.shape[1]/step )-1):
            x1 = i * step
            x2 = (i + 2) * step
            y1 = j * step
            y2 = (j + 2) * step
            line.append(img[x1:x2, y1:y2])
            if i < xx and j == yy:
                y1 = img.shape[1]-ISZ_
                y2 = img.shape[1]
                line.append(img[x1:x2, y1:y2])
                
        x = 2 * np.transpose(line, (0, 3, 1, 2)) - 1
        tmp = model_predict(model, x)
        print('single')
        tmpzzz = model_predict(model, x[0:1,:,:,:])
        if len(tmp)>img.shape[1]/step-1:
            last = tmp[-1]
            tmp = tmp[:-1]
        for j in range(tmp.shape[0]): #
            part1 = np.round(tmp[j],0) > 0
            part2 = np.round(prd[:, i * step:(i + 2) * step, j * step:(j + 2) * step],0) > 0 
            values = (part1 | part2).astype(np.float32)
            prd[:, i * step:(i + 2) * step, j * step:(j + 2) * step] = values
            if i < xx and j == yy:
                part1 = np.round(last,0) > 0
                part2 = np.round(prd[:, i * step:(i + 2) * step, img.shape[1]-ISZ_:img.shape[1]],0) > 0
                values = (part1 | part2).astype(np.float32)
                prd[:, i * step:(i + 2) * step, img.shape[1]-ISZ_:img.shape[1]] = values
                
                
        if i == xx:
            line = []
            for j in range(0, int(img.shape[1]/step)-1):
                x1 = img.shape[0]-ISZ_
                x2 = img.shape[0]
                y1 = j * step
                y2 = (j + 2) * step
                line.append(img[x1:x2, y1:y2])
                if j==yy:
                    y1 = img.shape[1]-ISZ_
                    y2 = img.shape[1]
                    line.append(img[x1:x2, y1:y2])
            x = 2 * np.transpose(line, (0, 3, 1, 2)) - 1
            tmp = model_predict(model, x)
            if len(tmp)>img.shape[1]/step-1:
                last = tmp[-1]
                tmp = tmp[:-1]
            for j in range(tmp.shape[0]): 
                part1 = np.round(tmp[j],0) > 0
                part2 = np.round(prd[:, img.shape[0]-ISZ_:img.shape[0], j * step:(j + 2) * step],0) > 0 
                values = (part1 | part2).astype(np.float32)
                prd[:, img.shape[0]-ISZ_:img.shape[0], j * step:(j + 2) * step] = values
                if j == yy:
                    part1 = np.round(last,0) > 0
                    part2 = np.round(prd[:, img.shape[0]-ISZ_:img.shape[0], img.shape[1]-ISZ_:img.shape[1]],0) > 0
                    values = (part1 | part2).astype(np.float32)                    
                    prd[:, img.shape[0]-ISZ_:img.shape[0], img.shape[1]-ISZ_:img.shape[1]] = values
                
                
    return prd[:, :img.shape[0], :img.shape[1]]


def compare_and_plot(img, msk_true, msk_pred, img_id='', save_to_disk=False):
    blues = 0
    img_and_msk_pred = img
    for i in range(img.shape[0]): # for every pixel:
        for j in range(img.shape[1]):
            if msk_pred[i,j] == 1 and msk_true[i,j] == 1:
                img_and_msk_pred[i,j] = (1, 1, 1)     # white
            elif msk_pred[i,j] == 0 and msk_true[i,j] == 1:    
                img_and_msk_pred[i,j] = (1, 0.1, 0.1) # red
            elif msk_pred[i,j] == 1 and msk_true[i,j] == 0:    
                img_and_msk_pred[i,j] = (0.1, 0.1, 1) # blue
                blues = blues + 1
    show_save_image(img_and_msk_pred, str(blues)+"_"+str(img_id), save_to_disk)           


def show_save_image(img, img_name=None, save_to_disk=False):        
    fig, ax1 = plt.subplots(figsize=(130,130),facecolor='w')
    if save_to_disk:
        directory = "AOI_train_125/predicted_data_v4/"+version_
        if img_name:
            file_name = "{directory}/{name}_machine_masks_img.png".format(name=img_name,
                                                              directory=directory)
        else:
            file_name = "{directory}/machine_masks_img.png".format(directory=directory)
        plt.imshow(img)
        fig.savefig(file_name, bbox_inches='tight')
        plt.close(fig)    
    else:
        plt.imshow(img)
        
def test_training_set():
    
    model = model_load_spacenet()
    img = np.load(Big_x % N_Cls + ".npy")[:img_size*5,:img_size*5]
    msk_true = np.load(Big_y % N_Cls + ".npy")[:img_size*5,:img_size*5]
    
    msk_true = msk_true[:,:,0]
    msk_true = np.round(msk_true, 0)

    true_msk_count_no_thres, true_msk_count = calculate_mask_count(msk_true)
    print('Number of true masks before threshold: ',true_msk_count_no_thres) 
    print('Number of true masks after threshold: ',true_msk_count)

    msk_pred = predict_test_image_spacenet(img, model)
    msk_pred = np.squeeze(msk_pred) 
    msk_pred = np.round(msk_pred, 0)
    pred_msk_count_no_thres, pred_msk_count = calculate_mask_count(msk_pred)
    print('Number of prediction masks before threshold: ', pred_msk_count_no_thres) 
    print('Number of prediction masks after threshold: ', pred_msk_count)

    rez = jaccard_similarity_score(msk_true, msk_pred)
    print ("Jaccard koef from lib:", rez)
    
    
    rez2 = jaccard_coef(msk_true, msk_pred)
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        print("Our f-tion Jaccard koef  ", rez2.eval())

    print ("Searching masks on Prediction Big Mask")
    true_msk_count_no_thres2, true_msk_count2 = compare_every_mask(msk_pred, msk_true)
    print ("Reversed. Searching masks on True Big Mask")
    true_msk_count_no_thres2, true_msk_count2 = compare_every_mask(msk_true, msk_pred)

#@timeit
def test_model():
    model = model_load_spacenet()
    img = np.load(Big_test_x % N_Cls + ".npy")
    msk_true = np.load(Big_test_y % N_Cls + ".npy")
    msk_true = msk_true[:,:,0]
    tf_msk_true = tf.convert_to_tensor(msk_true, np.float64)
    msk_true = np.round(msk_true, 0)

    true_msk_count_no_thres, true_msk_count = calculate_mask_count(msk_true)
    print('Number of true masks before threshold: ',true_msk_count_no_thres) 
    print('Number of true masks after threshold: ',true_msk_count)

    msk_pred = predict_test_image_spacenet_sliding(img, model)

    msk_pred = np.squeeze(msk_pred)
    tf_msk_pred = tf.convert_to_tensor(msk_pred, np.float64)
    msk_pred = np.round(msk_pred, 0)
    msk_pred = remove_bad_pixels_from_mask(msk_pred)
    pred_msk_count_no_thres, pred_msk_count = calculate_mask_count(msk_pred)
    print('Number of prediction masks before threshold: ', pred_msk_count_no_thres) 
    print('Number of prediction masks after threshold: ', pred_msk_count)

    rez = jaccard_similarity_score(msk_true, msk_pred)
    print ("Jaccard koef from lib:", rez)

    rez2 = jaccard_coef(msk_true, msk_pred)
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        print("Our f-tion Jaccard koef  ", rez2.eval())
    rez3 = jaccard_coef_int(tf_msk_true, tf_msk_pred)
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)
        print("Our f-tion Jaccard koef int ", rez3.eval())
    
    print ("Searching masks on Prediction Big Mask")
    true_msk_count_no_thres2, true_msk_count2 = compare_every_mask(msk_pred, msk_true)
    print ("Reversed. Searching masks on True Big Mask")
    true_msk_count_no_thres2, true_msk_count2 = compare_every_mask(msk_true, msk_pred)
    
def save_failed_images(img, model, mask_true):
    ISZ_ = int(ISZ)
    xx = int((img.shape[0]/ISZ_)-1)
    yy = int((img.shape[1]/ISZ_)-1)
    
    for i in range(0, int(img.shape[0]/ISZ_)):
        line = []
        for j in range(0, int(img.shape[1]/ISZ_)):
            x1 = i * ISZ_
            x2 = (i + 1) * ISZ_
            y1 = j * ISZ_
            y2 = (j + 1) * ISZ_
            small_img = img[x1:x2, y1:y2]
            line.append(small_img)
            if i < xx and j == yy:
                y1 = img.shape[1]-ISZ_
                y2 = img.shape[1]
                line.append(img[x1:x2, y1:y2])
                
        x = 2 * np.transpose(line, (0, 3, 1, 2)) - 1
        tmp = model.predict(x, batch_size=4)
        if len(tmp)>img.shape[1]/ISZ:
            last = tmp[-1]
            tmp = tmp[:-1]
        for j in range(tmp.shape[0]): #
            img_id = "i_{i}_j_{j}_".format(i=i, j=j)
            s_img = img[i * ISZ_:(i + 1) * ISZ_, j * ISZ_:(j + 1) * ISZ_]
            s_mask = mask_true[i * ISZ_:(i + 1) * ISZ_, j * ISZ_:(j + 1) * ISZ_]
            t_mask = tmp[j][0,:,:]
            t_mask = np.round(t_mask, 0)
            jacc = jaccard_similarity_score(s_mask, t_mask)
            
            if jacc < 0.6:
                print(img_id, jacc)
                img_id = img_id + "_jacc_" + "{0:.4f}".format(jacc) 
                img_id2 = img_id + "_real_"
                show_save_image(s_img, img_id2, save_to_disk=True) 
                compare_and_plot(s_img, s_mask, t_mask, img_id=img_id, save_to_disk=True)
            if i < xx and j == yy:
                img_id = "i_{i}_j_{j}_last_".format(i=i, j=j)
                s_img = img[i * ISZ_:(i + 1) * ISZ_, img.shape[1]-ISZ_:img.shape[1]]
                s_mask = mask_true[i * ISZ_:(i + 1) * ISZ_, img.shape[1]-ISZ_:img.shape[1]]
                t_mask = last[0,:,:]
                t_mask = np.round(t_mask, 0)
                jacc = jaccard_similarity_score(s_mask, t_mask)
                if jacc < 0.6:
                    img_id = img_id + "_jacc_" + "{0:.4f}".format(jacc)
                    img_id2 = img_id + "_real_"
                    show_save_image(s_img, img_id2, save_to_disk=True)
                    compare_and_plot(s_img, s_mask, t_mask, img_id=img_id, save_to_disk=True)
                    
                    


## Store patches to file

In [ ]:
if __name__ == '__main__':
    img = np.load(Big_x % N_Cls +".npy")
    msk = np.load(Big_y % N_Cls +".npy")
  
    x_trn, y_trn = get_patches_spacenet(img, msk, amt=25000, show_image=False) 
    np.savez(patches_file_x, x=x_trn)
    np.savez(patches_file_y, y=y_trn)

## Combine all images into 1 BIG image and save it to disk 
(performe only once for new images)

In [ ]:
if __name__ == '__main__':
    stick_all_train_spacenet_v2()


## Create training set
(We randomly take small images from 1 BIG image)

In [ ]:
if __name__ == '__main__':
    make_val_spacenet()
    print('done')   

## Train model and save it

In [ ]:
model = train_net_spacenet()
model_save_spacenet(model)
print('done')  
